## SARiSC Laboratorium

In [2]:
import pandas as pd
# from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler
import m2cgen as m2c

In [3]:
def GetTimeSeriesData(df: pd.DataFrame, target_col: str, point_per_series: int, forward_index: int, step_next: int=5):
    listNew = []
    listValue = []
    data_range = point_per_series
    data_range_index = data_range - 1
    full_range = point_per_series + forward_index
    full_range_index = full_range - 1
    start = 0
    print(df.__len__())
    for j in range(start, df.__len__() - full_range_index, step_next):
        listTemp = []
        for k in range(0, data_range):
            listTemp.append(df.iloc[k+j][df.columns.difference([target_col])].tolist())
            # listTemp.append(df[j + (k - start)])
        listNew.append(listTemp)
        listValue.append((df.iloc[full_range_index+j][target_col] - df.iloc[data_range_index+j][target_col]) >= 0)
        # listValue.append((tab[j + stepForward][targetIndes] >= tab[j][targetIndes]).astype(int))
    return np.array(listNew), np.array(listValue)

In [4]:
df = pd.read_csv('EURPLN_Candlestick_1_h_BID_01.05.2021-30.04.2024.csv', parse_dates=['Gmt time'],
                 date_parser=lambda x: pd.to_datetime(x, format='%d.%m.%Y %H:%M:%S.%f'))


# Dodaj kolumnę z dniem tygodnia
df['Day of Week'] = df['Gmt time'].dt.dayofweek

# Dodaj kolumnę z godziną w ciągu dnia
df['Hour of Day'] = df['Gmt time'].dt.hour

df = df.drop(columns=['Gmt time'])

C:\Users\Cezary\AppData\Local\Temp\ipykernel_25804\3584169503.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('EURPLN_Candlestick_1_h_BID_01.05.2021-30.04.2024.csv', parse_dates=['Gmt time'],


In [5]:
print(df.head())
columns_to_standardize = ['Open', 'High', 'Low', 'Close']
df_standarized = df.copy()
# Inicjalizuj StandardScaler
scaler = StandardScaler()

# Standaryzuj wybrane kolumny
df_standarized[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

# Dla kolumny 'Volume' podziel wartości przez odchylenie standardowe
volume_std = df['Volume'].std()
df_standarized['Volume'] = df['Volume'] / volume_std

print(df_standarized.head())

      Open     High      Low    Close  Volume  Day of Week  Hour of Day
0  4.55005  4.55005  4.55005  4.55005     0.0            5            0
1  4.55005  4.55005  4.55005  4.55005     0.0            5            1
2  4.55005  4.55005  4.55005  4.55005     0.0            5            2
3  4.55005  4.55005  4.55005  4.55005     0.0            5            3
4  4.55005  4.55005  4.55005  4.55005     0.0            5            4
       Open    High       Low     Close  Volume  Day of Week  Hour of Day
0 -0.134024 -0.1494 -0.117971 -0.134211     0.0            5            0
1 -0.134024 -0.1494 -0.117971 -0.134211     0.0            5            1
2 -0.134024 -0.1494 -0.117971 -0.134211     0.0            5            2
3 -0.134024 -0.1494 -0.117971 -0.134211     0.0            5            3
4 -0.134024 -0.1494 -0.117971 -0.134211     0.0            5            4


In [6]:
time_series_data, time_series_Value = GetTimeSeriesData(df_standarized, 'Open', 10, 2, 1)
print(time_series_data.shape)
print(time_series_Value.shape)

26304
(26293, 10, 6)
(26293,)


In [17]:
reshaped_array = time_series_data.reshape(time_series_data.shape[0], -1)
print(reshaped_array.shape)

(26293, 60)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(reshaped_array, time_series_Value, test_size=0.2, random_state=42)

In [19]:
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Przewidywanie na zbiorze testowym
y_pred = model.predict(X_test)

# Ocena modelu
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.30823350446853015


In [21]:
java_code = m2c.export_to_java(model)

# Zapisanie kodu Java do pliku
with open("LinearRegressionModel.java", "w") as f:
    f.write(java_code)

Mean Squared Error (MSE) dla regresji liniowej: 0.5558915986952425
Mean Squared Error (MSE) dla drzewa decyzyjnego: 0.25320457943178676
Mean Squared Error (MSE) dla modelu opartego na sieciach neuronowych: 0.6313990874886894


Mean Absolute Error (MAE):
Regresja liniowa: 0.5332001304956989
Drzewo decyzyjne: 0.32685366124031023
Sieć neuronowa: 0.602894055920684

Root Mean Squared Error (RMSE):
Regresja liniowa: 0.7455813830127751
Drzewo decyzyjne: 0.5031943753976059
Sieć neuronowa: 0.7946062468220908

R^2 score:
Regresja liniowa: 0.5757877060324521
Drzewo decyzyjne: 0.8067743859846794
Sieć neuronowa: 0.5181663908192364
